In [1]:
# with open('data/all.txt','r') as datafile:
#     quotes = datafile.readlines()
# list(sorted(set(quotes[0].split())))[:145]
# words = list(sorted(set(quotes[0].split())))[138:]
# words[0] = '0' 
# words[1] = 'a'
# word_to_int = dict([(w,i) for i,w in enumerate(words)])
# int_to_word = dict([(i,w) for i,w in enumerate(words)])
# word_to_int, int_to_word
# len(int_to_word)

In [1]:
# import re
# strip_special_chars = re.compile("[^A-Za-z0-9., ]+")

# def cleanSentences(string):
#     string = string.lower().replace("<br />", " ")
#     sentence = re.sub(strip_special_chars, "", string.lower())
#     return re.sub(r"(\w)([.,])", r"\1 \2", sentence)


# sentence = 'I will never be an old man. To me, old age is always 15 years older than I am.'

# print(cleanSentences(sentence))

# import pandas as pd
# data = pd.read_csv('quotes_all.csv',delimiter=';')
# topics = ['death' , 'family', 'freedom' , 'funny', 'happiness', 'life' , 'love', 'politics', 'success', 'science']

# for topic in topics:
#     quotes_data = data[data['Topic'].isin([topic])]

#     with open('%s-punctuation.txt'%topic,'w+') as quotefile:
#         for quote in quotes_data['Quote']:
#             quotefile.writelines(cleanSentences(quote.lower()) + " ")

# quotes_data = data[data['Topic'].isin(topics)]

# with open('all-punctuation.txt','w+') as quotefile:
#     for quote in quotes_data['Quote']:
#         quotefile.writelines(cleanSentences(quote.lower()) + " ")

In [2]:
import numpy as np
import sys
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.callbacks import LambdaCallback
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
with open('data/all-punctuation.txt','r') as quotefile:
    quotes = quotefile.readlines()

In [ ]:
with open('data/death.txt','r') as deathfile:
    deathquotes = deathfile.readlines()

In [78]:
t = Tokenizer(filters='')
t.fit_on_texts(quotes)
vocab_size = len(t.word_index) + 1

In [81]:
vocab_size, sorted(t.word_counts)

(13668,
 [',',
  ',000',
  ',000seat',
  ',400',
  '.',
  '.,',
  '...',
  '.4',
  '.5',
  '.a',
  '.c',
  '.d',
  '.j',
  '.k',
  '.m',
  '.outside',
  '.r',
  '.s',
  '.t',
  '.that',
  '.u',
  '0',
  '04',
  '1',
  '10',
  '100',
  '101',
  '102',
  '105',
  '11',
  '12',
  '125',
  '13',
  '132',
  '137',
  '14',
  '15',
  '16',
  '1600',
  '17',
  '17th',
  '18',
  '180',
  '1832',
  '187',
  '18karat',
  '18th',
  '19',
  '1911',
  '1937',
  '1948',
  '1950',
  '1950s',
  '1960',
  '1960s',
  '1965',
  '1967',
  '1969',
  '1970',
  '1970s',
  '1980s',
  '1984',
  '1988',
  '1989',
  '19th',
  '19thcentury',
  '19year',
  '1st',
  '1yearold',
  '2',
  '20',
  '200',
  '2000',
  '2001',
  '2008',
  '200million',
  '2010',
  '2025',
  '20million',
  '20s',
  '20th',
  '20thcentury',
  '21',
  '215',
  '21stcentury',
  '22',
  '22year',
  '24',
  '247',
  '25',
  '250',
  '26',
  '28',
  '3',
  '30',
  '300',
  '30th',
  '31',
  '32',
  '33',
  '37',
  '3d',
  '3yearold',
  '4',
  '4

In [82]:
index_word = dict([(i,w) for (w,i) in t.word_index.items()])

In [83]:
np.save('index_word-punc.npy',index_word)

In [ ]:
# integer encode the documents
encoded_docs = t.texts_to_sequences(deathquotes)
print(encoded_docs[0][:15])

In [84]:
embeddings_index = np.load('data/embeddings_index.npy')

In [85]:
embeddings_index = embeddings_index.item()

In [86]:
embeddings_index.get('...'), t.word_index['...']

(array([ -1.08139999e-01,   3.58909994e-01,   8.85179996e-01,
         -4.43589985e-01,  -5.94860017e-01,   3.36030006e-01,
          2.20670000e-01,  -2.28680000e-01,   3.17539990e-01,
         -3.80080014e-01,   6.47570014e-01,  -1.06610000e-01,
          9.07329991e-02,   4.26490009e-02,   2.32319999e-03,
         -8.50799978e-02,  -7.65439987e-01,   6.59240007e-01,
         -1.16489995e+00,   7.28209972e-01,   2.49039993e-01,
          5.30610010e-02,  -3.93390000e-01,  -2.65689999e-01,
          5.13100028e-01,   7.51519978e-01,  -1.60030007e-01,
         -4.55449998e-01,   5.58340013e-01,  -1.86780006e-01,
          1.38100004e-03,   4.25550014e-01,   1.31119996e-01,
         -2.21279994e-01,  -1.83630005e-01,  -4.20720018e-02,
          1.64600000e-01,   8.76670033e-02,  -3.97680001e-03,
         -5.45130014e-01,  -1.36419997e-01,   5.08280005e-03,
          1.05789997e-01,  -3.45759988e-01,  -5.86000025e-01,
         -8.03650022e-01,   8.87759961e-03,  -1.88800007e-01,
        

In [87]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [89]:
embedding_matrix[136]

array([ -1.08139999e-01,   3.58909994e-01,   8.85179996e-01,
        -4.43589985e-01,  -5.94860017e-01,   3.36030006e-01,
         2.20670000e-01,  -2.28680000e-01,   3.17539990e-01,
        -3.80080014e-01,   6.47570014e-01,  -1.06610000e-01,
         9.07329991e-02,   4.26490009e-02,   2.32319999e-03,
        -8.50799978e-02,  -7.65439987e-01,   6.59240007e-01,
        -1.16489995e+00,   7.28209972e-01,   2.49039993e-01,
         5.30610010e-02,  -3.93390000e-01,  -2.65689999e-01,
         5.13100028e-01,   7.51519978e-01,  -1.60030007e-01,
        -4.55449998e-01,   5.58340013e-01,  -1.86780006e-01,
         1.38100004e-03,   4.25550014e-01,   1.31119996e-01,
        -2.21279994e-01,  -1.83630005e-01,  -4.20720018e-02,
         1.64600000e-01,   8.76670033e-02,  -3.97680001e-03,
        -5.45130014e-01,  -1.36419997e-01,   5.08280005e-03,
         1.05789997e-01,  -3.45759988e-01,  -5.86000025e-01,
        -8.03650022e-01,   8.87759961e-03,  -1.88800007e-01,
        -4.31169987e-01,

In [90]:
# np.save('word_index.npy',t.word_index)
np.save('embedding_matrix_punc.npy',embedding_matrix)

In [91]:
embedding_matrix = np.load('embedding_matrix_punc.npy')
embedding_matrix.shape

(13668, 100)

## Generate Sequences, Next Sequences and Classes

## Death Doc

In [ ]:
maxlen = 100
step = 1
seq_death = []
next_seq_death = []

In [ ]:
death_doc = encoded_docs[0]
quote_len_death = len(death_doc)

In [ ]:
for i in range(0, quote_len_death - maxlen, step):
    seq_death.append(death_doc[i: i + maxlen])
    next_seq_death.append(death_doc[i + maxlen])

print('nb sequences:', len(seq_death))

seq_death = np.asarray(seq_death)
next_seq_death = np.asarray(next_seq_death)

In [ ]:
labels_death = np.zeros(next_seq_death.shape)

In [ ]:
labels_death

## Funny Doc

In [13]:
with open('data/funny.txt','r') as funnyfile:
    funnyquotes = funnyfile.readlines()

In [14]:
encoded_docs = t.texts_to_sequences(funnyquotes)
funny_doc = encoded_docs[0]

In [15]:
maxlen = 100
step = 1
seq_funny = []
next_seq_funny = []

quote_len_funny = len(funny_doc)

In [16]:
for i in range(0, quote_len_funny - maxlen, step):
    seq_funny.append(funny_doc[i: i + maxlen])
    next_seq_funny.append(funny_doc[i + maxlen])

print('nb sequences:', len(seq_funny))

seq_funny = np.asarray(seq_funny)
next_seq_funny = np.asarray(next_seq_funny)

nb sequences: 24396


In [17]:
labels_funny = np.ones(next_seq_funny.shape)

In [18]:
labels_funny

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

## Combine

In [ ]:
X = np.vstack([seq_death,seq_funny])

In [ ]:
X.shape

In [ ]:
labels_death = np.reshape(labels_death,(labels_death.shape[0],1))
labels_funny  = np.reshape(labels_funny,(labels_funny.shape[0],1))

In [ ]:
y = np.vstack([labels_death,labels_funny])

In [ ]:
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

# Sequence Classification Keras

In [ ]:
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen, trainable=False)

In [ ]:
model = Sequential()
model.add(e)

In [ ]:
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))

In [ ]:
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
#model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=24)
'''Train on 38108 samples, validate on 9528 samples
Epoch 1/10
38108/38108 [==============================] - 80s 2ms/step - 
loss: 0.0391 - acc: 0.9859 - val_loss: 4.4262e-04 - val_acc: 0.9999'''

# Text Generation using Word Embeddings

In [19]:
seq_funny, next_seq_funny

(array([[    6,   343,    77, ...,    57,  1419,    40],
        [  343,    77,     5, ...,  1419,    40,    42],
        [   77,     5,    41, ...,    40,    42,  1038],
        ..., 
        [   73,    13, 10360, ...,  3394,  1798,    23],
        [   13, 10360,     8, ...,  1798,    23,     1],
        [10360,     8,     1, ...,    23,     1,    53]]),
 array([  42, 1038,   75, ...,    1,   53, 1239]))

In [20]:
from keras.utils import to_categorical

In [21]:
y = to_categorical(next_seq_funny, num_classes=vocab_size)

In [22]:
X = seq_funny

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [38]:
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen, trainable=False)

In [39]:
model = Sequential()
model.add(e)

In [40]:
# model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))

In [41]:
#model.add(LSTM(100, return_sequences=True)) #
model.add(LSTM(100))
model.add(Dense(y.shape[1]))

In [42]:
model.add(Activation('softmax'))
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          1366200   
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 13662)             1379862   
_________________________________________________________________
activation_2 (Activation)    (None, 13662)             0         
Total params: 2,826,462
Trainable params: 1,460,262
Non-trainable params: 1,366,200
_________________________________________________________________
None


In [43]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [44]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), 
          epochs=10, 
          batch_size=24, #64
          callbacks=[print_callback])

Train on 19516 samples, validate on 4880 samples
Epoch 1/10
19516/19516 [==============================] - 130s 7ms/step - loss: 6.7342 - acc: 0.0378 - val_loss: 6.4484 - val_acc: 0.0402

----- Generating text after Epoch: 0
----- diversity: 0.2
[52, 108, 1641, 37, 1341, 37, 10038, 475, 8, 11, 10, 154, 776, 9, 2493, 11, 52, 2, 13, 115, 4923, 64, 10, 348, 11, 37, 9, 99, 475, 228, 102, 7, 360, 7, 139, 13, 22, 2949, 22, 3149, 10039, 7, 59, 245, 863, 4010, 7, 17, 96, 3149, 35, 15, 286, 1679, 280, 4, 7, 38, 250, 10, 9, 598, 5, 1336, 4, 53, 23, 6, 114, 6535, 227, 3, 601, 4, 61, 1, 83, 11, 5, 53, 93, 10, 38, 178, 61, 100, 4, 126, 12, 890, 23, 15, 4686, 2701, 40, 11, 133, 824, 7, 290]
----- Generating with seed: ['has', 'some', 'dramatic', 'or', 'violent', 'or', 'sinister', 'stuff', 'in', 'it', 'you', 'cant', 'forget', 'that', 'primarily', 'it', 'has', 'to', 'be', 'even', 'funnier', 'than', 'you', 'read', 'it', 'or', 'that', 'other', 'stuff', 'doesnt', 'work', 'i', 'wish', 'i', 'could', 'be', 

KeyboardInterrupt: 

In [32]:
index_word = np.load('index_word.npy')
index_word = index_word.item()

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    # preds = preds[1:]
    # preds = np.asarray(preds).astype('float64')
    # preds = np.log(preds) / temperature
    # exp_preds = np.exp(preds)
    # preds = exp_preds / np.sum(exp_preds)
    # probas = np.random.multinomial(1, preds, 1)
    return np.argmax(preds)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = np.random.randint(0, len(funny_doc) - maxlen - 1)
    for diversity in [0.2]: # 0.5, 1.0, 1.2
        print('----- diversity:', diversity)

        generated = ''
        sentence = funny_doc[start_index: start_index + maxlen]
        print(sentence)
        generated.join([str([index_word[value]]).join(' ') for value in sentence])
        print('----- Generating with seed: %s'%[index_word[word] for word in sentence])
        #sys.stdout.write(generated)

        for i in range(20):
            x_pred = np.reshape(sentence,(1, maxlen))

            preds = model.predict(x_pred, verbose=0)
            preds = preds[0]
            # print(preds.shape)
            next_index = sample(preds, diversity)
            #print(next_index)
            next_char = index_word[next_index]

            generated.join(str(next_char))
            sentence = np.append(sentence[1:],next_index)

            sys.stdout.write(next_char)
            sys.stdout.write(" ")
            sys.stdout.flush()
        print()